In [ ]:
from gasp3.gd.terrain.grs import make_dem

GRASS_WORKSPACE  = '/home/jasp/mdt/dems'
ELEVATION_DATA   = '/home/jasp/mdt/countours/countours_v2_lmt_dem_1.shp'
ELEVATION_COLUMN = 'data'
OUT_DEM          = '/home/jasp/mdt/dems/dem_1.tif'
EXTENT_TEMPLATE  = '/home/jasp/mdt/lmt_rst/lmt_dem_1.tif'
MASK             = '/home/jasp/mdt/rst_masks/masks_1.tif'
METHOD           = "CONTOUR"

import datetime as dt

time_a = dt.datetime.now().replace(microsecond=0)

make_dem(
    GRASS_WORKSPACE, ELEVATION_DATA, ELEVATION_COLUMN,
    OUT_DEM, EXTENT_TEMPLATE, method=METHOD, mask=MASK
)

time_b = dt.datetime.now().replace(microsecond=0)
print(time_b - time_a)

In [ ]:
"""
Produce a MDT for each cell in vec frid file
"""

alti = r'C:\gwork\fireloc\datasets\mdt_m888\alti_pt25k_3763.shp'
ref  = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'
outFld = r'C:\gwork\fireloc\datasets\mdt_m888'
out_lmt = r'C:\gwork\fireloc\datasets\ref_parts'
out_extra_lmt = r'C:\gwork\fireloc\datasets\mdt_m888\lmt_dem'
cell_id = 'cellid'

cell_width = 3000
cell_height= 3000

import os; import datetime as dt
from gasp3.fm          import tbl_to_obj
from gasp3.g.gop.prox  import df_buffer_extent
from gasp3.gt.prop.prj import get_epsg_shp
from gasp3.gt.to.shp import coords_to_boundary
from gasp3.pyt.oss import mkdir
from gasp3.gt.gop.ovlay import clip
from gasp3.pyt.oss import fld_exists

epsg = get_epsg_shp(ref)

grid_df = tbl_to_obj(ref)

grid_df = df_buffer_extent(grid_df, epsg, cell_width, mantainOriginalGeom=True)

In [ ]:
from gasp3.gt.to.shp import df_to_shp

df_to_shp(grid_df, r'C:\gwork\fireloc\datasets\ref\pt_ref_grid_bf.shp')

In [ ]:
time_a = dt.datetime.now().replace(microsecond=0)

for idx, row in grid_df.iterrows():
    # Get Lmt Shape
    left, bottom, right, top = row.old_geom.bounds
    lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        out_lmt, '{}_{}.shp'.format(os.path.basename(out_lmt), row[cell_id])
    ))
    
    # Get Extra Lmt
    left, bottom, right, top = row.geometry.bounds
    extra_lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        out_extra_lmt, '{}_{}.shp'.format(os.path.basename(out_extra_lmt), row[cell_id])
    ))

time_b = dt.datetime.now().replace(microsecond=0)
print(time_b - time_a)

In [ ]:
print(epsg)

In [ ]:
def clip_r(row):
    fld_path = os.path.join(outFld, 'data_' + str(row[cell_id]))
    
    isFld = fld_exists(fld_path)
    if not isFld:
        fld = mkdir(fld_path)
    else:
        return row
    
    # Get Lmt Shape
    left, bottom, right, top = row.old_geom.bounds
    lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        fld, 'lmt_{}.shp'.format(str(row[cell_id]))
    ))
    
    # Get Extra Lmt
    left, bottom, right, top = row.geometry.bounds
    extra_lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        fld, 'extra_lmt_{}.shp'.format(str(row[cell_id]))
    ))
    
    # Clip Altimetry
    alti_clp = clip(alti, extra_lmt, os.path.join(
        fld, 'alti_{}.shp'.format(str(row[cell_id]))
    ), api_gis="ogr2ogr")
    
    return row

time_a = dt.datetime.now().replace(microsecond=0)
grid_df.apply(lambda x: clip_r(x), axis=1)
time_b = dt.datetime.now().replace(microsecond=0)
print(time_b - time_a)

In [ ]:
"""
Run DEM tool using multiprocessing
"""

lmt_fld       = '/home/jasp/mdt/lmt_rst'
countours_fld = '/home/jasp/mdt/countours'
dem_fld       = '/home/jasp/mdt/dems'
elv_fld       = 'data'
masks         = '/home/jasp/mdt/rst_masks'

from gasp3.gd.terrain.grs import thrd_dem

thrd_dem(
    countours_fld, lmt_fld, dem_fld, elv_fld,
    refFormat='.tif', countoursFormat='.shp', demFormat='.tif',
    cellsize=10, masksFolder=masks, masksFormat='.tif'
)